# import libraries 

In [2]:
import nltk
#to open csv file
import csv
import pandas as pd
import numpy as np
#sentences & words tokenization
from nltk.tokenize import sent_tokenize, word_tokenize
#regular expression 
import re
#for stopwords
from nltk.corpus import stopwords
import string
#n-grams
from nltk.util import ngrams
#Lemmatization & Stemming
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [3]:
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [4]:
#for preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# csv path

In [6]:
dataset_path = r'C:\Users\DELL\OneDrive\code\py\nlp\NERdataset\NER-dataset.csv'

In [7]:
dataset = pd.read_csv(dataset_path, encoding="ISO-8859-1")
dataset.head()
#UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 25560: invalid start byte

,Sentence,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


# number of sentences and words

In [8]:
#number of sentences 
sentences = dataset['Sentence'].count()

#number of words
words = dataset['Word'].count()

print(f"Number of sentences: {sentences}")
print(f"Number of words: {words}")

Number of sentences: 460
Number of words: 10075


# number of stopwords and punctuation

In [9]:
stop_words = set(stopwords.words('english'))
#punctuation = set(string.punctuation)
punctuation = set('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')

In [10]:
num_stopwords = 0
num_punctuation = 0

for i in dataset['Word']:
    if i in stop_words:
        num_stopwords += 1
    elif i in string.punctuation:
        num_punctuation += 1

In [11]:
print("Number of stopwords:", num_stopwords)
# print(stop_words)
print("Number of punctuation:", num_punctuation)
# print(punctuation)

Number of stopwords: 3202
Number of punctuation: 800


# n-grams

In [12]:
# n = 3 
# ngrams = []

In [13]:
# for i in range(len(dataset['Word']) - n + 1):
#     ngram = ' '.join(dataset['Word'][i:i+n])
#     ngrams.append(ngram)

# for ngram in ngrams:
#     print([ngram])

# preprocessing dataset

## Lowercasing:
#### Convert all text to lowercase. This ensures that the model doesn't treat "Apple" and "apple" as different entities.

In [14]:
clean_dataset=[]

In [15]:
dataset['Word'] = dataset['Word'].str.lower()
dataset['Word']

# clean_dataset.extend(dataset['Word'].tolist())
# print(clean_dataset)

0            thousands
1                   of
2        demonstrators
3                 have
4              marched
             ...      
10070             more
10071             than
10072            4,500
10073            years
10074                .
Name: Word, Length: 10075, dtype: object

## Tokenization:
#### Tokenize your text into words or subword units. This breaks down the text into individual units, making it easier for the model to understand.

In [16]:
def tokenize_text(word):
    return word_tokenize(word)

dataset['Tokenized'] = dataset['Word'].apply(tokenize_text)
dataset = dataset[dataset['Word'] != '']
dataset[['Word', 'Tokenized']]
# clean_dataset = dataset['Tokenized'].tolist()
# print(clean_dataset)

,Word,Tokenized
0,thousands,[thousands]
1,of,[of]
2,demonstrators,[demonstrators]
3,have,[have]
4,marched,[marched]
...,...,...
10070,more,[more]
10071,than,[than]
10072,"4,500","[4,500]"
10073,years,[years]


## Removing Stopwords:
#### Remove common stopwords as they usually do not carry much information for NER tasks.

In [17]:
stop_words = set(stopwords.words('english'))
#punctuation = set(string.punctuation)
punctuation = set('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')

In [18]:
#remove stopwords and replace with an empty string
def remove_stopwords(word):

    return word if word not in stop_words else ''

dataset['Word'] = dataset['Word'].apply(remove_stopwords)
dataset['Word']
# dataset = dataset[dataset['Word'] != '']
# clean_dataset = dataset['Word'].tolist()
# print(clean_dataset)

0            thousands
1                     
2        demonstrators
3                     
4              marched
             ...      
10070                 
10071                 
10072            4,500
10073            years
10074                .
Name: Word, Length: 10075, dtype: object

## Removing Punctuation:
#### Depending on your task, you might want to remove or replace punctuation. In NER, punctuation may not contribute much to identifying entities.

In [19]:
def remove_punctuation(word):

    return word if word not in punctuation else ''

dataset['Word'] = dataset['Word'].apply(remove_punctuation)
dataset['Word']
# dataset = dataset[dataset['Word'] != '']
# clean_dataset = dataset['Word'].tolist()
# print(clean_dataset)

0            thousands
1                     
2        demonstrators
3                     
4              marched
             ...      
10070                 
10071                 
10072            4,500
10073            years
10074                 
Name: Word, Length: 10075, dtype: object

## Lemmatization or Stemming:
#### Reduce words to their base or root form. This helps in reducing dimensionality and treating different forms of a word as the same.

## Lemmatization

In [20]:
def lemmatize_word(word):
    if word is not None:
        lemmatizer = WordNetLemmatizer()
        return lemmatizer.lemmatize(word)
    
dataset['Lemmatized'] = dataset['Word'].apply(lemmatize_word)
dataset[['Word','Lemmatized']]
# dataset = dataset[dataset['Word'] != '']
# clean_dataset = dataset['Lemmatized'].tolist()
# print(clean_dataset)

,Word,Lemmatized
0,thousands,thousand
1,,
2,demonstrators,demonstrator
3,,
4,marched,marched
...,...,...
10070,,
10071,,
10072,"4,500","4,500"
10073,years,year


### preprocessing function 

In [21]:
def preprocess_text(text):
    # Apply lowercase
    text = text.lower()
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Remove punctuation
    punctuation = set('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
    tokens = [token for token in tokens if token not in punctuation]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# clean dataset

In [22]:
dataset['clean_dataset']=dataset['Lemmatized']

# Feature Extraction

In [23]:
# dataset['POS'] = dataset.POS.map({'NNP': 1})
def map_pos(v):
    return 1 if v == 'NNP' else 0  #we will target COUNTRIES only

dataset['POS'] = dataset['POS'].apply(map_pos)
dataset['POS']

0        0
1        0
2        0
3        0
4        0
        ..
10070    0
10071    0
10072    0
10073    0
10074    0
Name: POS, Length: 10075, dtype: int64

In [24]:
word_counts = {}

for index, row in dataset.iterrows():
    words = dataset['clean_dataset']
    
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

for word, count in word_counts.items():
    print(f"{word}: {count}")

thousand: 7
: 4236
demonstrator: 1
marched: 2
london: 8
protest: 8
war: 14
iraq: 16
demand: 3
withdrawal: 1
british: 14
troop: 17
country: 28
family: 2
soldier: 9
killed: 29
conflict: 3
joined: 1
protester: 3
carried: 3
banner: 1
slogan: 2
bush: 7
number: 5
one: 22
terrorist: 4
stop: 5
bombing: 10
house: 6
parliament: 4
rally: 1
hyde: 1
park: 1
police: 22
put: 6
marcher: 1
10,000: 1
organizer: 3
claimed: 4
1,00,000: 1
come: 5
eve: 1
annual: 6
conference: 3
britain: 3
's: 98
ruling: 1
labor: 4
party: 9
southern: 9
english: 2
seaside: 1
resort: 1
brighton: 1
divided: 1
participation: 1
continued: 1
deployment: 1
8,500: 1
march: 3
came: 3
ahead: 3
anti-war: 1
today: 1
city: 13
including: 8
rome: 2
paris: 7
madrid: 1
international: 8
atomic: 1
energy: 6
agency: 13
hold: 4
second: 3
day: 10
talk: 18
vienna: 1
wednesday: 18
respond: 1
iran: 19
resumption: 2
low-level: 1
uranium: 3
conversion: 2
week: 22
restarted: 2
part: 7
process: 4
isfahan: 1
nuclear: 21
plant: 6
iranian: 3
official: 43
s

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer(ngram_range=(1,2), max_features=1000)
features = countvec.fit_transform(dataset['clean_dataset'])

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(dataset['clean_dataset'])

X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, dataset['POS'], test_size=0.2)

# Classifier
nb_model_tfidf = MultinomialNB()
nb_model_tfidf.fit(X_train_tfidf, y_train)

# Prediction
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)

# Accuracy
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print(accuracy_tfidf)

0.915136476426799


In [26]:
pd.DataFrame(features.toarray(),columns=countvec.vocabulary_.keys())

,thousand,marched,london,protest,war,iraq,demand,british,troop,country,...,sperling,oversees,administration,clinton,window,bosnian,serb,massacre,srebrenica,hague
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10070,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10073,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


# ML Classifier

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, dataset['POS'], test_size=0.2)

In [28]:
print(X_train.shape)
print(X_test.shape)

(8060, 1000)
(2015, 1000)


In [29]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
#training
model.fit(X_train, y_train)

MultinomialNB()

In [30]:
predicted = model.predict(X_test)

print(np.mean(predicted == y_test))

0.913151364764268


In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predicted)

0.913151364764268

# Evaluation Metrics

In [33]:
# Evaluate the model on the training data
train_predictions = model.predict(X_train)
print("Training Accuracy:", accuracy_score(y_train, train_predictions))

Training Accuracy: 0.9259305210918114


In [34]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predicted))

[[1755    0]
 [ 175   85]]


In [35]:
from sklearn.metrics import classification_report

In [36]:
print(classification_report(y_test,predicted))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1755
           1       1.00      0.33      0.49       260

    accuracy                           0.91      2015
   macro avg       0.95      0.66      0.72      2015
weighted avg       0.92      0.91      0.89      2015

